In [1]:
import json

training_Data = [
    {"prompt": "빨강이", "completion": "Data Scientist"},
    {"prompt": "주황이", "completion": "Developer"},
    {"prompt": "노랑이", "completion": "Developer"},
    {"prompt": "초록이", "completion": "Developer"},
    {"prompt": "파랑이", "completion": "Developer"},
    {"prompt": "검둥이", "completion": "Data Scientist"},
    {"prompt": "보랑이", "completion": "Developer"},
    {"prompt": "남둥이", "completion": "Developer"},
    {"prompt": "하늘이", "completion": "Data Scientist"},
    {"prompt": "개나리", "completion": "Data Scientist"}
]

In [9]:
# file_name = "traininig_data.jsonl" -> model id None으로 나옴
file_name = "training_data.json"

with open(file_name, "w", encoding="utf-8") as output_file:
    for entry in training_Data:
        json.dump(entry, output_file)
        output_file.write('\n')

In [10]:
from openai import OpenAI
import keyring

client = OpenAI(
    api_key=keyring.get_password('openai', 'key_for_windows')
)

# upload the file for fine-tuning
upload_response = client.files.create(
    file=open(file_name, 'rb'),
    purpose='fine-tune'
)

In [11]:
# we can check file_id in the website : https://platform.openai.com/storage/files/file-eLWUHnNWiR1ac3QS8ae2exlR
file_id = upload_response.id
file_id

'file-eLWUHnNWiR1ac3QS8ae2exlR'

In [12]:
# create a fine-tuned model
fine_tune_response = client.fine_tuning.jobs.create(
    training_file=file_id,
    model='davinci-002'
)
fine_tune_response

FineTuningJob(id='ftjob-nbMXXwjngOfJaYuvtik8XgFp', created_at=1723173023, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='davinci-002', object='fine_tuning.job', organization_id='org-YfLvFUJwemLm4m8qa44pA6ht', result_files=[], seed=686063928, status='validating_files', trained_tokens=None, training_file='file-eLWUHnNWiR1ac3QS8ae2exlR', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
# fine-tuning job list 10
client.fine_tuning.jobs.list(limit=10)

# chekc status of fine-tuning
client.fine_tuning.jobs.retrieve("ftjob-1Wz1uJv09RjKRIulunzgjeRN")

FineTuningJob(id='ftjob-1Wz1uJv09RjKRIulunzgjeRN', created_at=1723165770, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:davinci-002:personal::9u8UwG5j', finished_at=1723166061, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=16), model='davinci-002', object='fine_tuning.job', organization_id='org-YfLvFUJwemLm4m8qa44pA6ht', result_files=['file-I0JTLrji1EnHg8YebmfQTg0f'], seed=1560774168, status='succeeded', trained_tokens=600, training_file='file-69dbIoeUqpUsYUbGJ2a3eXNo', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [14]:
# list up 10 events in the fine-tuned job
fine_tune_events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=fine_tune_response.id)
fine_tune_events

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-WrrwPavVyWIzv9Bh4gGS01tA', created_at=1723173049, level='info', message='Fine-tuning job started', object='fine_tuning.job.event', data=None, type='message'), FineTuningJobEvent(id='ftevent-3xBMmOdJzxRwthBAsXCkIk2i', created_at=1723173046, level='info', message='Files validated, moving job to queued state', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-wV9sqGLHqKmhwbYQ8eutianC', created_at=1723173023, level='info', message='Validating training file: file-eLWUHnNWiR1ac3QS8ae2exlR', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-yZVm8XkevmS4H8foeJAyopHX', created_at=1723173023, level='info', message='Created fine-tuning job: ftjob-nbMXXwjngOfJaYuvtik8XgFp', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [16]:
# use fine-tuned model
completion = client.completions.create(
    model = 'ft:davinci-002:personal::9u8UwG5j',
    prompt=['주황이는?']
)
print(completion.choices[0].text)

 zDeveloperDeveloperDeveloperDeveloperDeveloperDeveloperDeveloperDeveloperDeveloperDeveloperDeveloperDeveloperDeveloperDeveloperDeveloper


In [15]:
# use fine-tuned model
completion = client.completions.create(
    model = 'ft:davinci-002:personal::9u8UwG5j',
    prompt=['개나리는?']
)
print(completion.choices[0].text)


Data Scientist ScientistDeveloper Data Scientist ScientistData Scientist ScientistData Scientist ScientistData Scientist
